# Training Script for SQL Edge Hands-On

To-Do
- Change values with &lt; &gt; to the appropriate settings. (Total 4 occurance)
- Make sure to upload ```traindat.csv``` in this directory
- Run each cells on Azure Machine Learning Notebook

Task 1 : Import required modules.

In [ ]:
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig

Task 2 : Set AML environments

In [ ]:
ws = Workspace(
    subscription_id="<YOUR SUBSCRIPTION ID>",
    resource_group="<YOUR RESOURCE GROUP NAME>",
    workspace_name="<YOUR AML WORKSPACE NAME>")

Task 3 : Run automated machine learning (AutoML) job

(This will take several minutes to complete.)

In [ ]:
import pandas as pd

experiment = Experiment(ws, "sqledge-handson-automl-onnx")
df = pd.read_csv("traindat.csv", header=0)
x_train = df.drop(columns=["grade"])
y_train = df["grade"]
automl_config = AutoMLConfig(
    task = "classification",
    iteration_timeout_minutes = 30,
    iterations = 10,
    primary_metric="AUC_weighted",
    training_data=df,
    label_column_name="grade",
    n_cross_validations=2,
    enable_onnx_compatible_models=True)
local_run = experiment.submit(automl_config, show_output = True)
best_run, model = local_run.get_output(return_onnx_model=True)

Task 4 : Register the generated onnx model into Azure SQL Edge

In [ ]:
import pyodbc

server = "<YOUR SQL EDGE SERVER IP>"
userid = "sa"
password = "P@ssw0rd"
database = "FoodInspection"

connection_string = "Driver={{ODBC Driver 17 for SQL Server}};Server={};Database={};UID={};PWD={};".format(server, database, userid, password)
conn = pyodbc.connect(connection_string, autocommit=True)
cursor = conn.cursor()
query = f"INSERT INTO Models ([data], [description]) VALUES (?,'grade column prediction onnx model')"
params  = (pyodbc.Binary(model.SerializeToString()))
cursor.execute(query, params)
conn.commit()
cursor.close()
conn.close()